# Moving Average Strategy Backtest

Moving average strategies are among the most popular technical analysis approaches. They smooth out price data to identify trends and generate buy/sell signals based on crossovers.

## Strategies Tested:

1. **SMA Crossover (50/200)**: Classic "Golden Cross" and "Death Cross" strategy
2. **EMA Crossover (12/26)**: More responsive exponential moving averages
3. **Triple MA (10/50/200)**: Confirms strong trends with three moving averages
4. **Adaptive MA (20/50)**: Adjusts behavior based on market volatility

These strategies work best in trending markets and can struggle in choppy, sideways conditions.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../..")

from utils.simulation import BacktestSimulation, TradingAction
from strategies.moving_average_strategy import (
    create_sma_crossover_strategy,
    create_ema_crossover_strategy,
    create_triple_ma_strategy,
    create_adaptive_ma_strategy
)
from datetime import datetime

# Configuration
ticker = "MSTR"
tickers = [ticker]
start = datetime(2024, 1, 1)
end = datetime(2024, 12, 31)
initial_cash = 100000

## 1. SMA Crossover Strategy (50/200)

The classic moving average strategy using 50-day and 200-day simple moving averages.
- **Golden Cross**: 50-day MA crosses above 200-day MA → BUY
- **Death Cross**: 50-day MA crosses below 200-day MA → SELL

In [9]:
# Create and run SMA crossover strategy
sma_strategy = create_sma_crossover_strategy(
    ticker=ticker,
    short_window=50,
    long_window=200
)

sim_sma = BacktestSimulation(
    tickers=tickers,
    start_date=start,
    end_date=end,
    initial_cash=initial_cash,
    strategy_callback=sma_strategy
)

results_sma = sim_sma.run()

print("\n" + "="*50)
print("SMA CROSSOVER STRATEGY (50/200) PERFORMANCE")
print("="*50)
sim_sma.print_performance_stats()

transactions_sma = sim_sma.get_transactions()
print(f"\nTotal transactions: {len(transactions_sma)}")
if len(transactions_sma) > 0:
    print("\nTransaction History:")
    print(transactions_sma.to_string())

sim_sma.plot_portfolio_history()

Loading market data...
Running simulation from 2024-01-02 to 2024-12-30
Total trading days: 251


/Users/ryan/Desktop/quantitative-trading-models/strategies/moving_average/../../utils/simulation.py:36: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=self.start_date, end=self.end_date, progress=False)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## 2. EMA Crossover Strategy (12/26)

Exponential moving averages give more weight to recent prices, making them more responsive.
Using shorter periods (12/26) for faster signals.

In [ ]:
# Create and run EMA crossover strategy
ema_strategy = create_ema_crossover_strategy(
    ticker=ticker,
    short_window=12,
    long_window=26
)

sim_ema = BacktestSimulation(
    tickers=tickers,
    start_date=start,
    end_date=end,
    initial_cash=initial_cash,
    strategy_callback=ema_strategy
)

results_ema = sim_ema.run()

print("\n" + "="*50)
print("EMA CROSSOVER STRATEGY (12/26) PERFORMANCE")
print("="*50)
sim_ema.print_performance_stats()

transactions_ema = sim_ema.get_transactions()
print(f"\nTotal transactions: {len(transactions_ema)}")
if len(transactions_ema) > 0:
    print("\nTransaction History:")
    print(transactions_ema.to_string())

sim_ema.plot_portfolio_history()

## 3. Triple Moving Average Strategy (10/50/200)

Uses three moving averages to confirm strong trends:
- **BUY**: When Fast (10) > Medium (50) > Slow (200) - strong uptrend
- **SELL**: When Fast (10) crosses below Medium (50) - weakening trend

In [ ]:
# Create and run triple MA strategy
triple_ma_strategy = create_triple_ma_strategy(
    ticker=ticker,
    fast_window=10,
    medium_window=50,
    slow_window=200
)

sim_triple = BacktestSimulation(
    tickers=tickers,
    start_date=start,
    end_date=end,
    initial_cash=initial_cash,
    strategy_callback=triple_ma_strategy
)

results_triple = sim_triple.run()

print("\n" + "="*50)
print("TRIPLE MA STRATEGY (10/50/200) PERFORMANCE")
print("="*50)
sim_triple.print_performance_stats()

transactions_triple = sim_triple.get_transactions()
print(f"\nTotal transactions: {len(transactions_triple)}")
if len(transactions_triple) > 0:
    print("\nTransaction History:")
    print(transactions_triple.to_string())

sim_triple.plot_portfolio_history()

## 4. Adaptive Moving Average Strategy (20/50)

An intelligent strategy that adjusts to market conditions:
- **Low Volatility**: Standard crossover signals
- **High Volatility**: Requires 3 days of confirmation to avoid false signals (whipsaws)

In [ ]:
# Create and run adaptive MA strategy
adaptive_strategy = create_adaptive_ma_strategy(
    ticker=ticker,
    short_window=20,
    long_window=50,
    volatility_threshold=0.02
)

sim_adaptive = BacktestSimulation(
    tickers=tickers,
    start_date=start,
    end_date=end,
    initial_cash=initial_cash,
    strategy_callback=adaptive_strategy
)

results_adaptive = sim_adaptive.run()

print("\n" + "="*50)
print("ADAPTIVE MA STRATEGY (20/50) PERFORMANCE")
print("="*50)
sim_adaptive.print_performance_stats()

transactions_adaptive = sim_adaptive.get_transactions()
print(f"\nTotal transactions: {len(transactions_adaptive)}")
if len(transactions_adaptive) > 0:
    print("\nTransaction History:")
    print(transactions_adaptive.to_string())

sim_adaptive.plot_portfolio_history()

## Buy and Hold Comparison

In [ ]:
# Compare with Buy and Hold strategy
def buy_and_hold(date, portfolio, market_data, actions):
    """Simple buy and hold strategy for comparison"""
    current_price = market_data['prices'][ticker]
    
    # Buy stock with all available cash on first day
    if ticker not in portfolio.positions or portfolio.positions[ticker].shares == 0:
        max_shares = int(portfolio.cash / current_price)
        if max_shares > 0:
            actions.buy_stock(portfolio, ticker, max_shares, current_price)

# Run buy and hold simulation
sim_bh = BacktestSimulation(
    tickers=tickers,
    start_date=start,
    end_date=end,
    initial_cash=initial_cash,
    strategy_callback=buy_and_hold
)

results_bh = sim_bh.run()

print("\n" + "="*50)
print("BUY AND HOLD STRATEGY PERFORMANCE")
print("="*50)
sim_bh.print_performance_stats()

sim_bh.plot_portfolio_history()

## Strategy Comparison and Analysis

In [ ]:
# Create comprehensive comparison
comparison_data = {
    'Strategy': ['SMA 50/200', 'EMA 12/26', 'Triple MA', 'Adaptive MA', 'Buy & Hold'],
    'Final Value': [
        results_sma['portfolio_value'].iloc[-1],
        results_ema['portfolio_value'].iloc[-1],
        results_triple['portfolio_value'].iloc[-1],
        results_adaptive['portfolio_value'].iloc[-1],
        results_bh['portfolio_value'].iloc[-1]
    ],
    'Total Return (%)': [
        ((results_sma['portfolio_value'].iloc[-1] / initial_cash) - 1) * 100,
        ((results_ema['portfolio_value'].iloc[-1] / initial_cash) - 1) * 100,
        ((results_triple['portfolio_value'].iloc[-1] / initial_cash) - 1) * 100,
        ((results_adaptive['portfolio_value'].iloc[-1] / initial_cash) - 1) * 100,
        ((results_bh['portfolio_value'].iloc[-1] / initial_cash) - 1) * 100
    ],
    'Transactions': [
        len(transactions_sma),
        len(transactions_ema),
        len(transactions_triple),
        len(transactions_adaptive),
        len(sim_bh.get_transactions())
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("\n" + "="*70)
print("MOVING AVERAGE STRATEGIES COMPARISON")
print("="*70)
print(comparison_df.to_string(index=False))

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Portfolio values over time
axes[0, 0].plot(results_sma.index, results_sma['portfolio_value'], label='SMA 50/200', linewidth=2, alpha=0.8)
axes[0, 0].plot(results_ema.index, results_ema['portfolio_value'], label='EMA 12/26', linewidth=2, alpha=0.8)
axes[0, 0].plot(results_triple.index, results_triple['portfolio_value'], label='Triple MA', linewidth=2, alpha=0.8)
axes[0, 0].plot(results_adaptive.index, results_adaptive['portfolio_value'], label='Adaptive MA', linewidth=2, alpha=0.8)
axes[0, 0].plot(results_bh.index, results_bh['portfolio_value'], label='Buy & Hold', linewidth=2, linestyle='--', alpha=0.8)
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Portfolio Value ($)')
axes[0, 0].set_title('Portfolio Value Over Time')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Final returns comparison
strategies = comparison_df['Strategy']
returns = comparison_df['Total Return (%)']
colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#06A77D']
bars = axes[0, 1].bar(range(len(strategies)), returns, color=colors)
axes[0, 1].set_xticks(range(len(strategies)))
axes[0, 1].set_xticklabels(strategies, rotation=45, ha='right')
axes[0, 1].set_ylabel('Total Return (%)')
axes[0, 1].set_title('Final Returns Comparison')
axes[0, 1].grid(True, alpha=0.3, axis='y')
axes[0, 1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    axes[0, 1].text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.1f}%',
                    ha='center', va='bottom' if height > 0 else 'top')

# 3. Transaction count comparison
transaction_counts = comparison_df['Transactions']
axes[1, 0].bar(range(len(strategies)), transaction_counts, color=colors)
axes[1, 0].set_xticks(range(len(strategies)))
axes[1, 0].set_xticklabels(strategies, rotation=45, ha='right')
axes[1, 0].set_ylabel('Number of Transactions')
axes[1, 0].set_title('Transaction Activity')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# 4. Drawdown analysis
for result, label, color in zip(
    [results_sma, results_ema, results_triple, results_adaptive, results_bh],
    ['SMA 50/200', 'EMA 12/26', 'Triple MA', 'Adaptive MA', 'Buy & Hold'],
    colors
):
    rolling_max = result['portfolio_value'].expanding().max()
    drawdown = (result['portfolio_value'] - rolling_max) / rolling_max * 100
    axes[1, 1].plot(result.index, drawdown, label=label, linewidth=2, alpha=0.8, color=color)

axes[1, 1].set_xlabel('Date')
axes[1, 1].set_ylabel('Drawdown (%)')
axes[1, 1].set_title('Drawdown Analysis')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].fill_between(result.index, drawdown, 0, alpha=0.1, color='red')

plt.tight_layout()
plt.show()

# Calculate and display additional metrics
print("\n" + "="*70)
print("DETAILED PERFORMANCE METRICS")
print("="*70)

for result, name in zip(
    [results_sma, results_ema, results_triple, results_adaptive, results_bh],
    ['SMA 50/200', 'EMA 12/26', 'Triple MA', 'Adaptive MA', 'Buy & Hold']
):
    # Calculate max drawdown
    rolling_max = result['portfolio_value'].expanding().max()
    drawdown = (result['portfolio_value'] - rolling_max) / rolling_max
    max_drawdown = drawdown.min() * 100
    
    # Calculate Sharpe ratio (simplified)
    daily_returns = result['portfolio_value'].pct_change().dropna()
    sharpe_ratio = (daily_returns.mean() / daily_returns.std()) * np.sqrt(252) if daily_returns.std() > 0 else 0
    
    print(f"\n{name}:")
    print(f"  Max Drawdown: {max_drawdown:.2f}%")
    print(f"  Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"  Volatility: {daily_returns.std() * np.sqrt(252) * 100:.2f}%")

## Signal Analysis for SMA Strategy

Let's visualize the moving averages and buy/sell signals.

In [ ]:
# Plot price with moving averages and signals
import yfinance as yf

# Download price data
price_data = yf.download(ticker, start=start, end=end, progress=False)

# Calculate moving averages
price_data['SMA_50'] = price_data['Close'].rolling(window=50).mean()
price_data['SMA_200'] = price_data['Close'].rolling(window=200).mean()

# Identify buy and sell signals from transactions
buy_dates = []
sell_dates = []
buy_prices = []
sell_prices = []

if len(transactions_sma) > 0:
    for _, row in transactions_sma.iterrows():
        if row['action'] == 'BUY_STOCK':
            buy_dates.append(row['date'])
            buy_prices.append(row['price'])
        elif row['action'] == 'SELL_STOCK':
            sell_dates.append(row['date'])
            sell_prices.append(row['price'])

# Create visualization
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True)

# Plot 1: Price and Moving Averages
ax1.plot(price_data.index, price_data['Close'], label='Price', linewidth=2, color='black', alpha=0.7)
ax1.plot(price_data.index, price_data['SMA_50'], label='50-day SMA', linewidth=2, color='blue', alpha=0.7)
ax1.plot(price_data.index, price_data['SMA_200'], label='200-day SMA', linewidth=2, color='red', alpha=0.7)

# Mark buy and sell signals
if buy_dates:
    ax1.scatter(buy_dates, buy_prices, color='green', marker='^', s=200, label='Buy Signal', zorder=5)
if sell_dates:
    ax1.scatter(sell_dates, sell_prices, color='red', marker='v', s=200, label='Sell Signal', zorder=5)

ax1.set_ylabel('Price ($)')
ax1.set_title(f'{ticker} Price with SMA 50/200 Crossover Signals')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: Portfolio Value
ax2.plot(results_sma.index, results_sma['portfolio_value'], linewidth=2, color='#2E86AB')
ax2.fill_between(results_sma.index, results_sma['portfolio_value'], initial_cash, 
                 where=(results_sma['portfolio_value'] >= initial_cash), 
                 color='green', alpha=0.2, label='Profit')
ax2.fill_between(results_sma.index, results_sma['portfolio_value'], initial_cash,
                 where=(results_sma['portfolio_value'] < initial_cash),
                 color='red', alpha=0.2, label='Loss')
ax2.axhline(y=initial_cash, color='black', linestyle='--', linewidth=1, label='Initial Capital')
ax2.set_xlabel('Date')
ax2.set_ylabel('Portfolio Value ($)')
ax2.set_title('Portfolio Value Over Time')
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print signal summary
print("\n" + "="*50)
print("SIGNAL SUMMARY")
print("="*50)
print(f"Total Buy Signals: {len(buy_dates)}")
print(f"Total Sell Signals: {len(sell_dates)}")
if buy_dates:
    print(f"\nBuy Signal Dates: {[d.strftime('%Y-%m-%d') for d in buy_dates]}")
if sell_dates:
    print(f"Sell Signal Dates: {[d.strftime('%Y-%m-%d') for d in sell_dates]}")